In [ ]:
import sys
sys.path.append("../")  

In [2]:
import torch
torch.manual_seed(42)
import torch.optim as optim
from torch import nn

import dhg
from dhg import Hypergraph

import hgp
from hgp.models import HGNNP
from hgp.loss import loss_bs_matrix
from hgp.utils import from_pickle_to_hypergraph
from hgp.function import StraightThroughEstimator

DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE

/home/ycq/work/graph-partition-with-gcn/.env-HGP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=1)

In [3]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 3

h_hyper_prmts["convlayers1"] = {"in_channels": 1330, "out_channels": 1024, "use_bn": False, "drop_rate": 0.4}
h_hyper_prmts["convlayers3"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers4"] = {"in_channels": 512, "out_channels": 512, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers5"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.3}

l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":128, "use_bn":True, "drop_rate":0.1}
l_hyper_prmts["linerlayer2"] = {"in_channels":128, "out_channels":64, "use_bn":True, "drop_rate":0.1}
l_hyper_prmts["linerlayer3"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0.1}
l_hyper_prmts["linerlayer4"] = {"in_channels":32, "out_channels":3, "use_bn":False, "drop_rate":0.1}


hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [ ]:
class Trainer(nn.Module):

    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

In [10]:
G = from_pickle_to_hypergraph("../data/cora")
edges, _ = G.e
G.num_e,G.num_v

(1413, 1330)

In [6]:

X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
X = torch.eye(n=G.num_v)
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)

for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

optim = optim.Adam(hgnn_trainer.parameters(), lr=4e-5, weight_decay=5e-8)
hgnn_trainer.optimizer = optim

In [7]:
hgnn_trainer.layers
for n,p in hgnn_trainer.named_parameters():
    print(n,p)

layers.0.layers.0.theta.weight Parameter containing:
tensor([[ 0.0104,  0.0207,  0.0186,  ..., -0.0215, -0.0219,  0.0268],
        [ 0.0240, -0.0171,  0.0055,  ..., -0.0157, -0.0238, -0.0217],
        [ 0.0026,  0.0239,  0.0012,  ..., -0.0055,  0.0034,  0.0106],
        ...,
        [-0.0011, -0.0259, -0.0078,  ..., -0.0077,  0.0194, -0.0233],
        [ 0.0131, -0.0226, -0.0065,  ..., -0.0174, -0.0225, -0.0032],
        [ 0.0023, -0.0008,  0.0026,  ..., -0.0176,  0.0216,  0.0036]],
       device='cuda:1', requires_grad=True)
layers.0.layers.0.theta.bias Parameter containing:
tensor([-0.0206,  0.0176, -0.0196,  ...,  0.0268, -0.0214, -0.0069],
       device='cuda:1', requires_grad=True)
layers.0.layers.1.theta.weight Parameter containing:
tensor([[-0.0145,  0.0305, -0.0148,  ...,  0.0248,  0.0019, -0.0260],
        [ 0.0257,  0.0139, -0.0047,  ..., -0.0202, -0.0243, -0.0012],
        [-0.0019,  0.0008, -0.0276,  ..., -0.0158,  0.0153,  0.0309],
        ...,
        [ 0.0165,  0.0181,  0

In [8]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
for epoch in range(7500):
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    # train
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    # validation
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 1092.68896484375
                , loss1: 999.71162109375
                , loss2: 92.97738647460938
in 10 epoch, average loss: 10704.05625
                , loss1: 9999.51171875
                , loss2: 704.5443359375
in 20 epoch, average loss: 10445.846875
                , loss1: 10002.98125
                , loss2: 442.866845703125
in 30 epoch, average loss: 10252.75546875
                , loss1: 10005.871875
                , loss2: 246.8821533203125
in 40 epoch, average loss: 10134.4359375
                , loss1: 10007.559375
                , loss2: 126.87652587890625
in 50 epoch, average loss: 10063.0703125
                , loss1: 10008.603125
                , loss2: 54.466497802734374
in 60 epoch, average loss: 10029.6125
                , loss1: 10008.93515625
                , loss2: 20.675538635253908
in 70 epoch, average loss: 10015.4078125
                , loss1: 10009.3765625
                , loss2: 6.031185913085937
in 80 epoch, average l

In [9]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

164